<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2009%20-%20LlamaIndex_RAG_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index==0.12.43 deeplake==4.2.10 openai==1.92.0 llama-index-vector-stores-deeplake==0.3.3 llama-index-llms-openai==0.4.7 jedi==0.19.2

In [ ]:
import os

# os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'
# os.environ['ACTIVELOOP_TOKEN'] = '<YOUR_ACTIVELOOP_API_KEY>'

from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['ACTIVELOOP_TOKEN'] = userdata.get('ACTIVELOOP_TOKEN')

# Prepare Indexes

In [ ]:
!mkdir -p 'data/1k/'
!wget 'https://github.com/idontcalculate/data-repo/blob/main/machine_to_end_war.txt' -O './data/1k/tesla.txt'
!wget 'https://github.com/idontcalculate/data-repo/blob/main/prodigal_chapter10.txt' -O './data/1k/web.txt'

## From VectorStore

In [ ]:
# Import updated LlamaIndex modules
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, Settings
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool, QueryEngineTool, ToolMetadata
from llama_index.core.agent.workflow import FunctionAgent

In [ ]:
# Configure global settings
Settings.llm = OpenAI(model="gpt-4.1-mini", temperature=0)

In [ ]:
# Load Tesla documents and create vector store index
tesla_docs = SimpleDirectoryReader(input_files=["./data/1k/tesla.txt"]).load_data()

# Setup DeepLake vector store
my_activeloop_org_id = "anais" # TODO : replace with your org_id
my_activeloop_dataset_name = "LlamaIndex_tesla_predictions"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create vector store and storage context
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index from documents
tesla_index = VectorStoreIndex.from_documents(tesla_docs, storage_context=storage_context)

## From Local Index

In [ ]:
# Load and create local index for webtext
webtext_docs = SimpleDirectoryReader(input_files=["./data/1k/web.txt"]).load_data()

from llama_index.core import load_index_from_storage

try:
    # Try to load the index if it exists
    storage_context = StorageContext.from_defaults(persist_dir="./storage/webtext")
    webtext_index = load_index_from_storage(storage_context)
    print("Loaded the pre-computed index.")
except:
    # Otherwise, generate the index
    webtext_index = VectorStoreIndex.from_documents(webtext_docs)
    webtext_index.storage_context.persist(persist_dir="./storage/webtext")
    print("Generated the index.")

## Create Query Enginges

In [ ]:
tesla_query_engine = tesla_index.as_query_engine(similarity_top_k=3)
webtext_query_engine = webtext_index.as_query_engine(similarity_top_k=3)

## Create the Tools

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=tesla_query_engine,
        metadata=ToolMetadata(
            name="tesla_1k",
            description=(
                "Provides information about Tesla's statements that refers to future times and predictions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=webtext_query_engine,
        metadata=ToolMetadata(
            name="webtext_1k",
            description=(
                "Provides information about tesla's life and biographical data. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]


## Define the Agent

In [ ]:
# Create FunctionAgent
agent = FunctionAgent(
    tools=query_engine_tools,
    llm=Settings.llm,
    verbose=False
)

In [ ]:
# Interactive chat with the agent
import asyncio

async def chat_with_agent():
    """Async function to interact with the agent"""
    print("Agent is ready! Type 'quit' to exit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        try:
            response = await agent.run(user_input)
            print(f"Agent: {response}")
        except Exception as e:
            print(f"Error: {e}")

In [ ]:
# To stop this chat enter quit in the the chat bot
await chat_with_agent()

## Agents with Tools

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

# Create function tools using the updated API
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)

# Create agent with function tools
function_tools = [multiply_tool, add_tool]

math_agent = FunctionAgent(
    tools=function_tools,
    llm=Settings.llm,
    verbose=False,
    system_prompt="You are a helpful math assistant. Use the provided tools to perform calculations."
)

In [ ]:
# Test the math agent
async def test_math_agent():
    """Test the math agent with some calculations"""

    # Test multiplication
    response1 = await math_agent.run("What's 12 multiplied by 22? Make sure to use tools")
    print(f"Multiplication result: {response1}")

    # Test addition
    response2 = await math_agent.run("What is 5 + 2?")
    print(f"Addition result: {response2}")

# Run the test
await test_math_agent()

In [ ]:
# Enhanced agent with both query engines and math tools
all_tools = query_engine_tools + function_tools

enhanced_agent = FunctionAgent(
    tools=all_tools,
    llm=Settings.llm,
    verbose=False,
    system_prompt=(
        "You are an AI assistant that can answer questions about Tesla and perform mathematical calculations. "
        "Use the tesla_1k tool for questions about Tesla's future predictions and statements. "
        "Use the webtext_1k tool for questions about Tesla's biographical information. "
        "Use the math tools (add, multiply) for mathematical calculations."
    )
)

# Multi-tool usage
async def demo_enhanced_agent():
    """Demonstrate the enhanced agent with multiple tools"""

    queries = [
        "What are Tesla's predictions for the future?",
        "Tell me about Tesla's early life",
        "Calculate 15 * 8 + 25"
    ]

    for query in queries:
        print(f"\nUser: {query}")
        response = await enhanced_agent.run(query)
        print(f"Agent: {response}")

# Run the demo
await demo_enhanced_agent()